In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import gc

In [2]:
total = pd.read_csv("output.csv")
cat_cols = ['team', 'conference']
total.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [3]:
# To conduct week by week inference, we will train our model pre-2025-26 season and inference on our 2025-26 weeks
total_pre = total[total['season'] != 2025]
total_inf = total[total['season'] == 2025] # our inference set

In [4]:
# Dropping rows where season is before 2001-02 season
total_pre = total_pre[total_pre['season'] > 2000]

# Neural Networks

While we achieved 100% accuracy for our Light GBM inferences, our model will eventually reach its baseline as mentioned above. Thus, we will explore another model to see if it can achieve a higher baseline accuracy: neural networks

In [ ]:
# Assign device to gpu if available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Training on Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Training on NVIDIA GPU (CUDA)")
else:
    device = torch.device("cpu")
    print("Training on CPU")

print(f"Using device: {device}")

Training on Apple Silicon GPU (MPS)
Using device: mps


### Data Preparation

As extensively shown in our Light GBM model, we need to keep our groups (season, week, conference) in tact when training our model. As such, we need to create a custom batch sampler that keeps entire groups together.

In [6]:
class GroupBatchSampler:
    def __init__(self, group_ids, batch_size):
        self.group_ids = group_ids
        self.batch_size = batch_size

        # Group indices by group_id
        self.groups = {}
        for idx, group_id in enumerate(group_ids):
            if group_id not in self.groups:
                self.groups[group_id] = []
            self.groups[group_id].append(idx)

        self.group_list = list(self.groups.keys())

    def __iter__(self):
        # Shuffle the order of the groups
        np.random.shuffle(self.group_list)

        batch = []
        for group_id in self.group_list:
            group_indices = self.groups[group_id]
            batch.extend(group_indices)

            # Yield batch when reching batch_size
            while len(batch) >= self.batch_size:
                yield batch[:self.batch_size]
                batch = batch[self.batch_size:]

         # Yield remaining samples   
        if len(batch) > 0:
            yield batch
    
    def __len__(self):
        return (len(self.group_ids) + self.batch_size - 1) // self.batch_size

Unlike our Light GBM model, we need to one-hot encode our categorical features (team, conference) for our neural net model.

In [ ]:
# From our data above
# Reminder: total_pre = data up to 2024-25 season
X = total_pre.drop(columns=['full_name', 'player_id', 'pow_player_id', 'player_of_the_week', 'won_player_of_the_week', 'all_star_this_season', 'mvp_this_season',
 'all_nba_first_team_this_season', 'all_nba_second_team_this_season', 'all_nba_third_team_this_season', 'week_start', 'pow_conference',
 'is_win_vs_over_500', 'opponent_has_all_nba'])

y = total_pre['won_player_of_the_week']

print(f"Number of features before one-hot encoding: {len(X.columns)}")

# One-hot encode
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# Rename conference one-hot encoding column: 'conference_West' -> 'conference'
X = X.rename(columns={'conference_West': 'conference'})

print(f"Number of features after one-hot encoding: {len(X.columns)}")

Number of features before one-hot encoding: 65
Number of features after one-hot encoding: 95


In [8]:
X.columns

Index(['season', 'week', 'games_played_this_week', 'numMinutes', 'points',
       'assists', 'blocks', 'steals', 'reboundsTotal', 'reboundsDefensive',
       'reboundsOffensive', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'threePointersAttempted', 'threePointersMade', 'freeThrowsAttempted',
       'freeThrowsMade', 'turnovers', 'foulsPersonal', 'plusMinusPoints',
       'wins_this_week', 'wins_vs_team_with_all_nba_player', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'team_pts', 'team_ast', 'team_blk',
       'team_stl', 'team_gms', 'fieldGoalsPercentage',
       'threePointersPercentage', 'freeThrowsPercentage', 'points_mean_season',
       'points_std_season', 'assists_mean_season', 'assists_st

In [14]:
len(X.columns)

95

Since we one-hot encoded our categorical features, the number of features will grow a lot due to the number of teams in the NBA. This is totally ok since neural networks can handle high dimensionality

### Neural Network Architecture

Our neural network has three layers that get progressively smaller (256 -> 128 -> 64 neurons), like a funnel that squeezes information down from ~100 input features into a single score per player. The first layer spots basic patterns in the data, the middle layer combines these patterns into bigger ideas like "great scorer" or "well-rounded player," and the final layer picks out what actually matters for winning player of the week. 

Between each layer, we use ReLU activation which lets the network learn that certain combinations of stats matter more together than separately (like how 35 points + 3 wins is way more impressive than just 35 points alone). We also randomly turn off 30% of neurons during training (dropout) so the network doesn't just memorize the data and actually learns real patterns. The final output is a ranking score rather than a probability because we just need to know who scores highest, not the exact chance they win.

In [9]:
class POTWRanker(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        super(POTWRanker, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        # Build hidden layers
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        # Output layer (single score per player)
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

### Ranking Loss Function

Our loss function teaches the model to make sure winners score higher than everyone else in the same week, rather than just predicting winners independently. For every week, it compares the winner to every non-winner (so if there's 1 winner and 50 other players, it makes 50 comparisons) and gives the model a penalty whenever the winner's score isn't at least 1.0 points higher. This is like telling the model "the winner needs to clearly beat everyone else, not just barely edge them out." The key advantage is that the model learns what makes one player better than another in that specific week's context, rather than trying to learn some universal "good player" score. This only works if we keep entire weeks together during training. If we mix players from different weeks, the comparisons don't make sense since those players never actually competed for the same award.

In [10]:
class PairwiseRankingLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(PairwiseRankingLoss, self).__init__()
        self.margin = margin
    
    def forward(self, scores, labels, group_indices):
        total_loss = torch.tensor(0.0, requires_grad=True)
        num_pairs = 0
        
        # For each group (week/conference)
        for group_id in torch.unique(group_indices):
            mask = group_indices == group_id
            group_scores = scores[mask].squeeze()
            group_labels = labels[mask]
            
            # Find winners and non-winners
            winner_mask = group_labels == 1
            non_winner_mask = group_labels == 0
            
            if winner_mask.sum() == 0 or non_winner_mask.sum() == 0:
                continue
            
            winner_scores = group_scores[winner_mask]
            non_winner_scores = group_scores[non_winner_mask]
            
            # Compute all pairwise differences at once
            score_diff = winner_scores.unsqueeze(1) - non_winner_scores.unsqueeze(0)
            
            # Apply margin and clamp
            losses = torch.clamp(self.margin - score_diff, min=0)
            
            total_loss = total_loss + losses.sum()  # Add to tensor
            num_pairs += losses.numel()
        
        # Return tensor (even if zero)
        if num_pairs == 0:
            return torch.tensor(0.0, requires_grad=True)
        
        return total_loss / num_pairs

### Cross-validation Training

Again, we will run time-based cross validation to prevent overfitting results swaying our decisions. We will follow the same format as our cross validation from Light GBM.

Unlike our Light GBM model, in our neural net model, we will scale our data because neural networks are sensitive to feature magnitudes—features. For example, points would dominate features like fieldGoalsPercentage during training, causing the model to learn poorly. Standardization ensures all features contribute equally to the model's learning process and helps gradient descent converge faster and more reliably. 

In [13]:
# Hyperparameters
epochs = 50
batch_size = 512
hidden_dims = [512, 256, 128, 64]
dropout = 0.2
lr = 0.001

unique_seasons = sorted(X['season'].unique())
folds = 3  # Reduced from 5 to 3 compared to Light GBM
k = [1, 3, 5, 10]
cv_results = []

for fold in range(1, folds + 1):
    val_season = unique_seasons[-fold]
    train_seasons = unique_seasons[:-fold]
    
    val_mask = X['season'] == val_season
    train_mask = X['season'].isin(train_seasons)
    
    X_train = X[train_mask].copy()
    y_train = y[train_mask].copy()
    X_val = X[val_mask].copy()
    y_val = y[val_mask].copy()

    print(f"Validation {fold}")
    print(f"Val Season: {val_season}")
    print(f"Train Seasons: {train_seasons[0]}-{train_seasons[-1]}")
    print(f"Train size: {len(X_train)}, Val size: {len(X_val)}")
    
    X_train['group_id'] = X_train.groupby(['season', 'week', 'conference']).ngroup()
    X_val['group_id'] = X_val.groupby(['season', 'week', 'conference']).ngroup()
    train_group_ids = X_train['group_id'].values
    val_group_ids = X_val['group_id'].values
    
    # Remove only metadata. This includes season and week. We will have our normalized 'z_' metrics capture temporal changes
    feature_cols = [col for col in X_train.columns 
                    if col not in ['group_id']]
    
    X_train_features = X_train[feature_cols].values
    X_val_features = X_val[feature_cols].values
    print(f"Train shape: {X_train_features.shape}")
    print(f"Val shape: {X_val_features.shape}")
    
    # Scaling our data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_features).astype(np.float32)
    X_val_scaled = scaler.transform(X_val_features).astype(np.float32)
    
    print("\nCreating PyTorch dataset...")
    train_dataset = TensorDataset(
        torch.from_numpy(X_train_scaled),
        torch.from_numpy(y_train.values.astype(np.float32)),
        torch.from_numpy(train_group_ids.astype(np.int64))
    )
    
    print("\nCreating GroupBatchSampler and DataLoader...")
    batch_sampler = GroupBatchSampler(train_group_ids, batch_size=batch_size)
    train_loader = DataLoader(train_dataset, batch_sampler=batch_sampler)
    print(f"DataLoader created. Number of batches: {len(train_loader)}")
    
    print("\nInitializing model...")
    input_dim = X_train_scaled.shape[1]
    model = POTWRanker(input_dim, hidden_dims=hidden_dims, dropout=dropout)
    print(f"Model created. Parameters: {sum(p.numel() for p in model.parameters()):,}")

    model = model.to(device)
    print(f"Model moved to {device}")
    
    print("\nSetting up loss and optimizer...")
    criterion = PairwiseRankingLoss(margin=1.0)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    print("Loss and optimizer ready")
    
    print(f"\nTraining {epochs} epochs")
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch_X, batch_y, batch_groups in train_loader:
            # Move batch data to GPU
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            batch_groups = batch_groups.to(device)

            optimizer.zero_grad()
            scores = model(batch_X)
            loss = criterion(scores, batch_y, batch_groups)
            loss.backward() # Compute gradients
            optimizer.step() # Update weights * learning rate (Gradient descent)
            
            total_loss += loss.item() # Track running total loss
        
        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    print("\nEvaluation")
    
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        X_val_tensor = torch.from_numpy(X_val_scaled).to(device)
        val_scores = model(X_val_tensor).cpu().squeeze().numpy()
    
    df_val = X_val[['season', 'week', 'conference']].copy().reset_index(drop=True)
    df_val['score'] = val_scores
    df_val['y_true'] = y_val.reset_index(drop=True).values
    
    k_dict = {i: [] for i in k}
    groups = df_val.groupby(['season', 'week', 'conference'], observed=True)
    ranks = []
    
    for _, group in groups:
        if group['y_true'].sum() == 0:
            continue
        
        group_sorted = group.sort_values('score', ascending=False).reset_index(drop=True)
        pos_idx = group_sorted.index[group_sorted['y_true'] == 1]
        
        for i in k:
            top_k = group_sorted.head(i)
            hit = top_k['y_true'].max()
            k_dict[i].append(hit)
        
        for idx in pos_idx:
            ranks.append(idx + 1)
    
    ranks = np.array(ranks)
    curr = {
        "Fold": fold,
        "Top_rank": ranks.min(),
        "Lowest_rank": ranks.max(),
        "Percentiles": np.percentile(ranks, [10, 25, 50, 75, 90])
    }
    for i, hits in k_dict.items():
        curr[f"Top_{i}_avg_hits"] = np.mean(hits)
    cv_results.append(curr)
    
    print(f"\nFold {fold} Results:")
    print(f"Top-1 Accuracy: {curr['Top_1_avg_hits']:.4f}")
    print(f"Top-3 Accuracy: {curr['Top_3_avg_hits']:.4f}")
    print(f"Top-5 Accuracy: {curr['Top_5_avg_hits']:.4f}")
    print(f"Top Rank: {curr['Top_rank']}")
    print(f"Lowest Rank: {curr['Lowest_rank']}")
    
    # Clean up memory after each fold to clear up RAM
    del model, optimizer, criterion, train_loader, train_dataset, batch_sampler
    del X_train, y_train, X_val, y_val
    del X_train_features, X_val_features, X_train_scaled, X_val_scaled
    del train_group_ids, val_group_ids
    gc.collect()

Validation 1
Val Season: 2024
Train Seasons: 2001-2023
Train size: 165504, Val size: 7971
  Number of features: 95
Train shape: (165504, 95)
Val shape: (7971, 95)

Creating PyTorch dataset...

Creating GroupBatchSampler and DataLoader...
DataLoader created. Number of batches: 324

Initializing model...
Model created. Parameters: 221,697
Model moved to mps

Setting up loss and optimizer...
Loss and optimizer ready

Training 50 epochs
Epoch 10/50, Loss: 0.0087
Epoch 20/50, Loss: 0.0050
Epoch 30/50, Loss: 0.0025
Epoch 40/50, Loss: 0.0021
Epoch 50/50, Loss: 0.0018

Evaluation

Fold 1 Results:
Top-1 Accuracy: 0.4000
Top-3 Accuracy: 0.6250
Top-5 Accuracy: 0.7250
Top Rank: 1
Lowest Rank: 22
Validation 2
Val Season: 2023
Train Seasons: 2001-2022
Train size: 158395, Val size: 7109
  Number of features: 95
Train shape: (158395, 95)
Val shape: (7109, 95)

Creating PyTorch dataset...

Creating GroupBatchSampler and DataLoader...
DataLoader created. Number of batches: 310

Initializing model...
Mod

In [16]:
# Final Results
cv_df = pd.DataFrame(cv_results)

for idx, row in cv_df.iterrows():
    print(f"Fold {row['Fold']}: Top-1={row['Top_1_avg_hits']:.4f}, "
          f"Top-5={row['Top_5_avg_hits']:.4f}, Top_rank={row['Top_rank']}, "
          f"Lowest_rank={row['Lowest_rank']}, Rank_Percentile={row['Percentiles']}")

print("\nAverage Performance Across Folds:")

for i in k:
    print(f"Top-{i} Accuracy: {cv_df[f'Top_{i}_avg_hits'].mean()}")

Fold 1: Top-1=0.4000, Top-5=0.7250, Top_rank=1, Lowest_rank=22, Rank_Percentile=[ 1.   1.   3.   6.  10.8]
Fold 2: Top-1=0.4857, Top-5=0.9143, Top_rank=1, Lowest_rank=18, Rank_Percentile=[1.  1.  2.  4.  6.2]
Fold 3: Top-1=0.5952, Top-5=0.8810, Top_rank=1, Lowest_rank=35, Rank_Percentile=[1.  1.  1.  3.  7.6]

Average Performance Across Folds:
Top-1 Accuracy: 0.4936507936507937
Top-3 Accuracy: 0.7511904761904762
Top-5 Accuracy: 0.8400793650793651
Top-10 Accuracy: 0.9412698412698411


In [ ]:
print("COMPARISON WITH LIGHTGBM")
print(f"LightGBM Top-1:        53.0%")
print(f"Neural Network Top-1:  {cv_df['Top_1_avg_hits'].mean():.1%}")
print(f"Difference:            {(cv_df['Top_1_avg_hits'].mean() - 0.53)*100:+.1f}%")